In [1]:
%load_ext autoreload
%autoreload 2

import os
import json
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

**Load in data**

In [ ]:
if not os.path.exists("./data/data.db"):
    os.system("python build_db.py")

conn = sqlite3.connect("./data/data.db")
c = conn.cursor()


COLS_EXCLUDE = set([
    "StateAbbr",
    "StateDesc",
    "CountyName",
    "CountyFIPS",
    "TractFIPS",
    "fips",
    "stateabb",
    "sedaadmin",
    "sedaadminname",
    "TRACT",
    "COUNT"
])

c.execute(f"""
WITH seda_tracts AS (
    SELECT *
    FROM seda s
    JOIN nces n
    ON s.sedaadmin = n.LEAID
    WHERE s.year = 2019
)

, food_atlas_tracts AS (
    SELECT *
    FROM food f
    JOIN seda_tracts st
    ON st.TRACT = f.CensusTract
)

, cdc_tracts AS (
    SELECT *
    FROM cdc c
    JOIN food_atlas_tracts ft
    ON ft.TRACT = c.TractFIPS
)

SELECT *
FROM cdc_tracts;
""")

rows = c.fetchall()
print(len(rows))
columns = [col[0] for col in c.description]

## Write the rows manually into a CSV file without pandas
with open("data/full_joined_table.csv", "w") as f:
    to_exclude = set()
    to_exclude.add(0) # Remove StateAbbr (duplicate)
    to_exclude.add(1) # Remove CountyName (duplicate)
    to_exclude.add(2) # Remove TractFIPS (duplicate)
    race_cols = []
    for i, col in enumerate(columns):
        if col in set(["stateabb", "MHLTH_CrudePrev:1", "SLEEP_CrudePrev:1", "fips", "LEAID", "NAME_LEA19", "TRACT", "COUNT"]): # Duplicates
            to_exclude.add(i)

    f.write(",".join([col for i, col in enumerate(columns) if i not in to_exclude]) + "\n")
    for row in tqdm(rows):
        # if row[0] == "CA":
        f.write(",".join([str(x) for i, x in enumerate(row) if i not in to_exclude]) + "\n")


995894


100%|██████████| 995894/995894 [00:13<00:00, 74536.94it/s]


In [4]:
df = pd.read_csv("data/full_joined_table.csv")
df.shape

(995894, 69)

In [ ]:
df.head()

,MHLTH_CrudePrev,SLEEP_CrudePrev,CensusTract,State,County,Urban,LATracts_half,LATracts10,PovertyRate,LowIncomeTracts,lahunvhalfshare,lahunv10share,stateabb,sedaadmin,sedaadminname,subject,grade,year,gcs_mn_all,gcs_mn_se_all,tot_asmt_all,gcs_mn_asn,gcs_mn_se_asn,tot_asmt_asn,gcs_mn_blk,gcs_mn_se_blk,tot_asmt_blk,gcs_mn_ecd,gcs_mn_se_ecd,tot_asmt_ecd,gcs_mn_fem,gcs_mn_se_fem,tot_asmt_fem,gcs_mn_hsp,gcs_mn_se_hsp,tot_asmt_hsp,gcs_mn_mal,gcs_mn_se_mal,tot_asmt_mal,gcs_mn_mfg,gcs_mn_se_mfg,tot_asmt_mfg,gcs_mn_nam,gcs_mn_se_nam,tot_asmt_nam,gcs_mn_nec,gcs_mn_se_nec,tot_asmt_nec,gcs_mn_neg,gcs_mn_se_neg,tot_asmt_neg,gcs_mn_wag,gcs_mn_se_wag,tot_asmt_wag,gcs_mn_wbg,gcs_mn_se_wbg,tot_asmt_wbg,gcs_mn_whg,gcs_mn_se_whg,tot_asmt_whg,gcs_mn_wht,gcs_mn_se_wht,tot_asmt_wht,gcs_mn_wng,gcs_mn_se_wng,tot_asmt_wng,last_bie,LANDAREA,WATERAREA
0,18.6,36.9,1095030701,Alabama,Marshall County,0,0,0,26.0,1,5.26,NaN,AL,100005,Albertville City,mth,3,2019,2.084955,0.17362,448,NaN,NaN,NaN,NaN,NaN,NaN,1.499328,0.216733,200.0,1.917122,0.203595,225.0,1.737057,0.197765,252.0,2.228891,0.219325,223.0,0.338979,0.299259,448.0,NaN,NaN,NaN,2.566329,0.20235,248.0,1.077884,0.296578,448.0,NaN,NaN,NaN,NaN,NaN,NaN,0.817592,0.312159,418.0,2.579603,0.241355,166.0,NaN,NaN,NaN,0,2.116012,0.000000
1,18.9,37.3,1095030702,Alabama,Marshall County,0,0,0,15.2,0,6.77,NaN,AL,100005,Albertville City,mth,3,2019,2.084955,0.17362,448,NaN,NaN,NaN,NaN,NaN,NaN,1.499328,0.216733,200.0,1.917122,0.203595,225.0,1.737057,0.197765,252.0,2.228891,0.219325,223.0,0.338979,0.299259,448.0,NaN,NaN,NaN,2.566329,0.20235,248.0,1.077884,0.296578,448.0,NaN,NaN,NaN,NaN,NaN,NaN,0.817592,0.312159,418.0,2.579603,0.241355,166.0,NaN,NaN,NaN,0,0.800889,0.009930
2,19.5,38.4,1095030801,Alabama,Marshall County,1,1,0,29.9,1,1.51,NaN,AL,100005,Albertville City,mth,3,2019,2.084955,0.17362,448,NaN,NaN,NaN,NaN,NaN,NaN,1.499328,0.216733,200.0,1.917122,0.203595,225.0,1.737057,0.197765,252.0,2.228891,0.219325,223.0,0.338979,0.299259,448.0,NaN,NaN,NaN,2.566329,0.20235,248.0,1.077884,0.296578,448.0,NaN,NaN,NaN,NaN,NaN,NaN,0.817592,0.312159,418.0,2.579603,0.241355,166.0,NaN,NaN,NaN,0,5.437714,0.012075
3,21.9,40.2,1095030802,Alabama,Marshall County,1,1,0,39.2,1,3.15,NaN,AL,100005,Albertville City,mth,3,2019,2.084955,0.17362,448,NaN,NaN,NaN,NaN,NaN,NaN,1.499328,0.216733,200.0,1.917122,0.203595,225.0,1.737057,0.197765,252.0,2.228891,0.219325,223.0,0.338979,0.299259,448.0,NaN,NaN,NaN,2.566329,0.20235,248.0,1.077884,0.296578,448.0,NaN,NaN,NaN,NaN,NaN,NaN,0.817592,0.312159,418.0,2.579603,0.241355,166.0,NaN,NaN,NaN,0,5.297315,0.017784
4,15.8,35.2,1095030902,Alabama,Marshall County,1,1,0,13.7,0,1.66,NaN,AL,100005,Albertville City,mth,3,2019,2.084955,0.17362,448,NaN,NaN,NaN,NaN,NaN,NaN,1.499328,0.216733,200.0,1.917122,0.203595,225.0,1.737057,0.197765,252.0,2.228891,0.219325,223.0,0.338979,0.299259,448.0,NaN,NaN,NaN,2.566329,0.20235,248.0,1.077884,0.296578,448.0,NaN,NaN,NaN,NaN,NaN,NaN,0.817592,0.312159,418.0,2.579603,0.241355,166.0,NaN,NaN,NaN,0,7.293396,0.033274


In [6]:
health_feature_cols = [
    "MHLTH_CrudePrev", # cont
    "SLEEP_CrudePrev", # cont
]

food_desert_cols = [
    "Urban",            # bool
    "LATracts_half",    # bool
    "LALTracts10",      # bool
    "PovertyRate",      # cont
    "LowIncomeTracts",  # bool
    "lahunvhalfshare",  # bool
    "lahunv10share",    # bool
]

academics_all_cols = [
    "gcs_mn_all", "gcs_mn_se_all", "tot_asmt_all",
    "gcs_mn_asn", "gcs_mn_se_asn", "tot_asmt_asn",
    "gcs_mn_blk", "gcs_mn_se_blk", "tot_asmt_blk",
    "gcs_mn_ecd", "gcs_mn_se_ecd", "tot_asmt_ecd",
    "gcs_mn_fem", "gcs_mn_se_fem", "tot_asmt_fem",
    "gcs_mn_hsp", "gcs_mn_se_hsp", "tot_asmt_hsp",
    "gcs_mn_mal", "gcs_mn_se_mal", "tot_asmt_mal",
    "gcs_mn_mfg", "gcs_mn_se_mfg", "tot_asmt_mfg",
    "gcs_mn_nam", "gcs_mn_se_nam", "tot_asmt_nam",
    "gcs_mn_nec", "gcs_mn_se_nec", "tot_asmt_nec",
    "gcs_mn_neg", "gcs_mn_se_neg", "tot_asmt_neg",
    "gcs_mn_wag", "gcs_mn_se_wag", "tot_asmt_wag",
    "gcs_mn_wbg", "gcs_mn_se_wbg", "tot_asmt_wbg",
    "gcs_mn_whg", "gcs_mn_se_whg", "tot_asmt_whg",
    "gcs_mn_wht", "gcs_mn_se_wht", "tot_asmt_wht",
    "gcs_mn_wng", "gcs_mn_se_wng", "tot_asmt_wng",
]

academics_race_cols = [
    "gcs_mn_all", "gcs_mn_se_all", "tot_asmt_all",
    "gcs_mn_asn", "gcs_mn_se_asn", "tot_asmt_asn",
    "gcs_mn_blk", "gcs_mn_se_blk", "tot_asmt_blk",
    "gcs_mn_hsp", "gcs_mn_se_hsp", "tot_asmt_hsp",
    "gcs_mn_nam", "gcs_mn_se_nam", "tot_asmt_nam",
]

academics_gender_cols = [
    "gcs_mn_all", "gcs_mn_se_all", "tot_asmt_all",
    "gcs_mn_fem", "gcs_mn_se_fem", "tot_asmt_fem",
    "gcs_mn_mal", "gcs_mn_se_mal", "tot_asmt_mal",
]